### User input

In [ ]:
decimal = '.'                                                                   # decimal of the input file
sep = ';'                                                                       # separator of the input file
input_dir = 'C:/Users/cace0002/AbspectroscoPY/results/df_mediancor_sw_30.0.csv' # input directory
output = 'C:/Users/cace0002/AbspectroscoPY/results/'                            # output directory
dateheadername = 'Timestamp'                                                    # header of the date  
header = 0                                                                      # header row number
dateparsingformat = '%Y-%m-%d %H:%M:%S'                                         # format of the date 
sample_name = 'sw'                                                              # name of the sample

S_f = 1                                                                         # sampling frequency

#To plot the time series of slope ratio, the user can modify:
col_sel = 'SR'                                                                  # column to plot
timestart = '2018-11-13 05:02:00'                                               # starting time and ending time
timeend =   '2018-12-04 08:44:00'                                
fig_format = '.tiff'                                                            # format of the exported figure
dpi = 300                                                                       # resolution of the exported figure

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from datetime import datetime
from scipy import interpolate
from sklearn.linear_model import LinearRegression

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### abs_slope_ratio

In [ ]:
def abs_slope_ratio(df_in,
                    sampling_frequency):
    '''
    function to get a dataframe with the slope values at shorter wavelengths (S275-295), at longer wavelengths (S350-400) and their ratio  (slope ratio)
    :argument df_in: dataframe in input
    :argument sampling_frequency: sampling frequency when computing the slope ratio
    :return: dataframe with S275-295, S350-400 and slope ratio (SR) at the sampling frequency specified
    '''
    df_in = df_in.reset_index()                                  # restore the original index as row
    df_in = df_in.T                                              # transpose the data frame  
    df_in = df_in.reset_index()                                  # restore the original index as row
    headers = df_in.iloc[0]                                      # rename the dates as headers                                     
    df_in = pd.DataFrame(df_in.values[1:], columns=headers)      # convert the first row to header
    df_in = df_in.rename(columns={"Timestamp": "wl"})            # rename the first column as "wl" = wavelength vector
    df_in['wl'] = df_in['wl'].replace({'nm':''},regex=True)      # remove "nm" from the wavelength vector
    wl = df_in['wl'].apply(pd.to_numeric)                        # convert the wavelength vector to numeric  

    iteration = len(df_in.columns)                               # number of loop iterations
    print('number of iterations',iteration)
    empty_matrix = np.zeros((iteration, 3))                      # create an empty matrix to fill in with the parameters s275_295, s350_400,SR for each datetime

    wl_275_295 = np.linspace(275,295,25)                         # create an array of 25 evenly spaced numbers over the specified interval
    wl_350_400 = np.linspace(350,400,25) 
    wl_275_295_resh = np.array(wl_275_295).reshape((-1, 1))      # reshape as column
    wl_350_400_resh = np.array(wl_350_400).reshape((-1, 1))

    counter = 0                                                  # use a counter in the loop to keep track of the iteration number
    for i in range(0,iteration,sampling_frequency): 
        counter = i                                           
        print(counter)                                 
        absorbance = df_in.iloc[:,i]                             # subset the absorbance values   
        sf = interpolate.interp1d(wl, absorbance,kind='cubic')# spline interpolation of third order to get as many absorbance values as the array of 25 wavelengths
        absorbance_275_295 = sf(wl_275_295)
        absorbance_350_400 = sf(wl_350_400)
        absorbance_275_295_log_resh = np.log(absorbance_275_295).reshape((-1, 1))
        absorbance_350_400_log_resh = np.log(absorbance_350_400).reshape((-1, 1))
        lm_275_295 = LinearRegression().fit(wl_275_295_resh, absorbance_275_295_log_resh) # perform a linear regression and calculate the slopes in the two wavelength ranges
        slope_275_295 = lm_275_295.coef_[0]
        lm_350_400 = LinearRegression().fit(wl_350_400_resh, absorbance_350_400_log_resh)
        slope_350_400 = lm_350_400.coef_[0]
        print('slope 275-295:',slope_275_295)
        print('slope 350-400:',slope_350_400)
        sr = slope_275_295/slope_350_400                         # compute the slope ratio 
        print('SR:',sr) 
        empty_matrix[i, 0] = slope_275_295                       # fill in the empty matrix with the calculated parameters
        empty_matrix[i, 1] = slope_350_400  
        empty_matrix[i, 2] = sr    

    sr_data = pd.DataFrame(empty_matrix, index=headers.iloc[0:], columns=['s275_295','s350_400','SR']) # create a dataframe with the calculated parameters
    #print(sr_data)
    df_out = sr_data[sr_data['SR'] != 0][1:]                     # keep only the rows in which the slope ratio is different from zero 
    df_out.index = pd.to_datetime(df_out.index)
    print(df_out)    
    return(df_out)

In [ ]:
df_sr = abs_slope_ratio(df, S_f)
df_sr[1:].to_csv(output + 'df_sr_' + str(sample_name) + '_' + str(S_f) +'.csv', index = True, sep = ';') # export the dataframe

In [ ]:
title = 'slope_ratio_'
abspy.makeaplot(df_sr, output, 'SR', timestart, timeend, sample_name, title, ylabel = 'Slope ratio [dimensionless]')